# Instagram Scraper



In [0]:
!pip install instaloader


  Created wheel for instaloader: filename=instaloader-4.2.10-cp36-none-any.whl size=41056 sha256=3257b554986bd6e50dcf06ddad5b86156bccb0646f785ad72d9eb873744a1c9b
  Stored in directory: /root/.cache/pip/wheels/16/78/4d/9693042fa92e1003fc2f87909e07d341c4242023cd9f16999f
Successfully built instaloader


In [0]:
import json
import instaloader
import profile 

In [0]:
import instaloader

L = instaloader.Instaloader()
USER = "natgeo"
PROFILE = instaloader.Profile.from_username(L.context, USER)

likes = []
comments = []
caption = []
url = []
i = 0
for post in PROFILE.get_posts():
    if i <=600 and post.is_video == False:
        likes.append(post.likes)
        comments.append(post.comments)
        caption.append(post.caption)
        url.append(post.url)
        i+=1
    else:
        pass


Too many queries in the last time. Need to wait 71 seconds, until 21:35.

Too many queries in the last time. Need to wait 73 seconds, until 21:36.

Too many queries in the last time. Need to wait 44 seconds, until 21:37.


In [0]:
import pandas as pd 
df = pd.DataFrame(list(zip(caption,url,likes,comments)))
df.columns=['caption','url','likes','comments']

In [0]:
import xlrd
from google.cloud import vision
import os
import pandas as pd

Application_Credentials = 'Python-03112e22a573.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.types.Image()

loc = ("image_url.xlsx")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)
df = pd.DataFrame()
# loop through every url, retreive the image and send to google vision
for i in range(sheet.nrows):
    image_src_temp = sheet.cell_value(i, 0)
    image.source.image_uri = image_src_temp
    response = client.label_detection(image=image)
    labels = response.label_annotations
    l = []
    for label in labels:
        l.append(label.description)
    s = ' '.join(l)
    print("s")
    print(s)
    df = df.append({'URL': image_src_temp, 'Labels': s}, ignore_index=True)
df.to_excel("Insta_withoutcomments1.xlsx",index=False)

# Google Vision API


In [0]:
!pip install --user google-cloud
!pip install --user google-cloud-vision
!pip install --upgrade google-cloud-storage

In [0]:
import xlrd
from google.cloud import vision
import os
import pandas as pd

In [0]:
Application_Credentials = "Text analysis-adf93dd6cd04.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.types.Image()

In [0]:
import pandas as pd
df1 = pd.read_csv('result.csv')
url = list(df1['url'])

In [0]:
df = pd.DataFrame()
for i in url:
    image.source.image_uri = i
    outcome = client.label_detection(image=image)
    labels = outcome.label_annotations
    text = []
    for label in labels:
        text.append(label.description)
    s = ' '.join(text)
    df = df.append({'URL': i, 'Labels': s}, ignore_index=True)


In [0]:
df.to_csv('lables.csv',sep=',')

# Task A


In [41]:
import pandas as pd
import csv
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
import nltk
import lda
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to C:\Users\Rui
[nltk_data]     Ying\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rui
[nltk_data]     Ying\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
df = pd.read_csv('result.csv')
df_tag = pd.read_csv('lables.csv')

In [43]:
df_merge = df.merge(df_tag, left_on= 'url',right_on ='URL')
del df_merge['URL']
df_merge.head()

,caption,url,likes,comments,Labels
0,"Photos by @hammond_robin | “I feel like, what ...",https://scontent-hkg3-2.cdninstagram.com/vp/13...,37942,169,Face Facial expression Wrinkle Forehead Skin H...
1,Photos by @carltonward | A large male panther ...,https://scontent-hkg3-2.cdninstagram.com/vp/63...,147833,411,Grass Pasture Wildlife Grass family Grassland ...
2,Photos by @brentstirton | // Supported by Nat ...,https://scontent-hkg3-2.cdninstagram.com/vp/fb...,286941,819,Adventure Recreation Fawn
3,Photo by Michael Yamashita @yamashitaphoto | L...,https://scontent-hkg3-2.cdninstagram.com/vp/5c...,400990,3050,Design Textile Colorfulness Pattern Photograph...
4,Photo by Erika Larsen @erikalarsen888 | Jacind...,https://scontent-hkg3-2.cdninstagram.com/vp/db...,151548,771,Hair Face Beauty Lip Lady Hairstyle Smile Eyeb...


In [44]:
def scale(x,cols):
    for col in cols:
        x[col]= x[col]/max(x[col])
    return x

In [45]:
df_merge = scale(df_merge,cols=['likes','comments'])
df_merge['engagement'] = 0.4*df_merge['likes']+0.6*df_merge['comments']

In [46]:
df_merge['engagement_dummy']=0
med = df_merge['engagement'].median()
for i in range(len(df_merge['engagement'])):
    if df_merge['engagement'][i] < med:
        df_merge['engagement_dummy'][i] = 0
    else:
        df_merge['engagement_dummy'][i] = 1

In [47]:
df_merge.head()

,caption,url,likes,comments,Labels,engagement,engagement_dummy
0,"Photos by @hammond_robin | “I feel like, what ...",https://scontent-hkg3-2.cdninstagram.com/vp/13...,0.015898,0.002960,Face Facial expression Wrinkle Forehead Skin H...,0.008135,0
1,Photos by @carltonward | A large male panther ...,https://scontent-hkg3-2.cdninstagram.com/vp/63...,0.061944,0.007199,Grass Pasture Wildlife Grass family Grassland ...,0.029097,0
2,Photos by @brentstirton | // Supported by Nat ...,https://scontent-hkg3-2.cdninstagram.com/vp/fb...,0.120233,0.014345,Adventure Recreation Fawn,0.056700,0
3,Photo by Michael Yamashita @yamashitaphoto | L...,https://scontent-hkg3-2.cdninstagram.com/vp/5c...,0.168021,0.053421,Design Textile Colorfulness Pattern Photograph...,0.099261,1
4,Photo by Erika Larsen @erikalarsen888 | Jacind...,https://scontent-hkg3-2.cdninstagram.com/vp/db...,0.063501,0.013504,Hair Face Beauty Lip Lady Hairstyle Smile Eyeb...,0.033503,0


# Lables Logistic Regression 


In [49]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
df_merge1 = df_merge.dropna()

df_all = pd.DataFrame()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(df_merge1['Labels'])
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
count_vector=cv.transform(df_merge1['Labels'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
for i in range(len(df_merge1['Labels'])):
    first_document_vector = tf_idf_vector[i]
    df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=[df_merge1.index.values[i]])
    df_all = pd.concat([df_all, df.T])


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_all, df_merge1['engagement_dummy'], test_size=0.3, random_state=1)

In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
c1=confusion_matrix(y_test, y_pred)
acc1=accuracy_score(y_test, y_pred)
print(accuracy_score(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames = ["Actual Class"], colnames = ["Predicted Class"])

0.7611111111111111


Predicted Class,0,1
Actual Class,,
0,67,20
1,23,70


# Caption Logistic Regression 


In [54]:
df_copy = df_merge.copy()
df_copy['cap'] = None
for i in range(len(df_copy['caption'])):
    w = str(df_copy['caption'][i]).lower().split('|')
    s = ' '.join(w[1:])
    df_copy['cap'][i] = s

In [55]:
df_copy = df_copy.dropna()

df_all_caption = pd.DataFrame()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(df_copy['cap'])
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
count_vector=cv.transform(df_copy['cap'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
for i in range(len(df_copy['caption'])):
    first_document_vector = tf_idf_vector[i]
    df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=[df_merge1.index.values[i]])
    df_all_caption = pd.concat([df_all_caption, df.T])

In [56]:
X1_train, X1_test, y1_train, y1_test = train_test_split(df_all_caption, df_merge1['engagement_dummy'], test_size=0.3, random_state=1)

In [112]:
model1 = LogisticRegression()
model1.fit(X1_train, y1_train)
y_pred1 = model1.predict(X1_test)
c2=confusion_matrix(y1_test, y_pred1)
acc2=accuracy_score(y1_test, y_pred1)
print(accuracy_score(y1_test, y_pred1))
pd.crosstab(y1_test, y_pred1, rownames = ["Actual Class"], colnames = ["Predicted Class"])

0.7166666666666667


Predicted Class,0,1
Actual Class,,
0,61,26
1,25,68


# Logistic Regression with both

In [58]:
df_both = df_merge.copy()
df_both['both'] = None
for i in range(len(df_both['caption'])):
    w = str(df_both['caption'][i]).lower().split('|')
    s = '  '.join(w[1:])
    df_both['both'][i] = s + str(df_merge['Labels'][i])

In [60]:
df_both = df_both.dropna()

df_final = pd.DataFrame()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(df_both['both'])
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
count_vector=cv.transform(df_both['both'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
for i in range(len(df_both['both'])):
    first_document_vector = tf_idf_vector[i]
    df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=[df_merge1.index.values[i]])
    df_final = pd.concat([df_final, df.T])

In [61]:
X2_train, X2_test, y2_train, y2_test = train_test_split(df_final, df_merge1['engagement_dummy'], test_size=0.3, random_state=1)

In [113]:
model2 = LogisticRegression()
model2.fit(X2_train, y2_train)
y_pred2 = model2.predict(X2_test)
c3=confusion_matrix(y2_test, y_pred2)
acc3=accuracy_score(y2_test, y_pred2)
print(accuracy_score(y2_test, y_pred2))
pd.crosstab(y2_test, y_pred2, rownames = ["Actual Class"], colnames = ["Predicted Class"])

0.7833333333333333


Predicted Class,0,1
Actual Class,,
0,65,22
1,17,76


# Topic Modelig

In [63]:
!pip install lda

In [64]:
df_merge = df_merge.dropna()
w_token=RegexpTokenizer(r'\w+')
w_lemma = WordNetLemmatizer()
stop =set(stopwords.words('english'))

def tokenize(version_desc):
    text = w_lemma.lemmatize(version_desc.lower())
    tokens = w_token.tokenize(text)
    return tokens

vec = CountVectorizer(tokenizer=tokenize, stop_words=stop, decode_error='ignore')
words = vec.fit_transform(df_merge["Labels"])

model = lda.LDA(n_topics = 5, n_iter = 1000, random_state = 1)
model.fit(words)

INFO:lda:n_documents: 598
INFO:lda:vocab_size: 1063
INFO:lda:n_words: 6725
INFO:lda:n_topics: 5
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -63731
INFO:lda:<10> log likelihood: -44391
INFO:lda:<20> log likelihood: -42603
INFO:lda:<30> log likelihood: -42096
INFO:lda:<40> log likelihood: -41687
INFO:lda:<50> log likelihood: -41489
INFO:lda:<60> log likelihood: -41359
INFO:lda:<70> log likelihood: -41216
INFO:lda:<80> log likelihood: -41120
INFO:lda:<90> log likelihood: -40979
INFO:lda:<100> log likelihood: -41048
INFO:lda:<110> log likelihood: -41034
INFO:lda:<120> log likelihood: -41013
INFO:lda:<130> log likelihood: -41030
INFO:lda:<140> log likelihood: -40995
INFO:lda:<150> log likelihood: -41064
INFO:lda:<160> log likelihood: -40964
INFO:lda:<170> log likelihood: -40954
INFO:lda:<180> log likelihood: -40926
INFO:lda:<190> log likelihood: -40869
INFO:lda:<200> log likelihood: -40833
INFO:lda:<210> log likelihood: -40795
INFO:lda:<220> log likelihood: -40916
INFO:lda:<230> log 

In [65]:
topic_word = model.topic_word_
df_topic = pd.DataFrame(model.doc_topic_)
df_merge = df_merge.join(df_topic)

In [66]:
df_final =pd.DataFrame()
for i in range(5):
    topic = str(i)
    df_final[topic]=df_merge.groupby(["Labels"])[i].mean()

df_final = df_final.reset_index()
topics = pd.DataFrame(topic_word)
topics.columns = vec.get_feature_names()
topics = topics.T

In [67]:
df_final.to_csv("Label_Topic.csv")

In [68]:
topics.to_csv("topics.csv")

In [69]:
topics.sort_values(0, ascending=False).head(10)

,0,1,2,3,4
sea,0.062352,0.000008,0.000007,0.020636,0.000007
fish,0.048924,0.000008,0.000007,0.000006,0.000007
marine,0.047006,0.000008,0.000007,0.000006,0.000007
turtle,0.040292,0.000008,0.000007,0.000006,0.000007
ice,0.037415,0.000008,0.000007,0.000006,0.000007
water,0.034538,0.000008,0.000007,0.060645,0.000007
shark,0.034538,0.000008,0.000007,0.000006,0.000007
underwater,0.032619,0.000008,0.000007,0.000006,0.000007
biology,0.031660,0.000008,0.000007,0.000006,0.000007
organism,0.025906,0.000008,0.000007,0.000006,0.011214


In [70]:
topics.columns = ["marine","landscape","human","nature","wildlife"]

In [71]:
df_engagement = df_final.merge(df_merge[["Labels","engagement"]], on = "Labels")

In [72]:
top_25 = df_engagement.sort_values("engagement", ascending=False)[:150]
top_25.columns = ["Labels","marine","landscape","human","nature","wildlife", "engagement"]

In [73]:
bottom_25 = df_engagement.sort_values("engagement", ascending=False)[-150:]
bottom_25.columns = ["Labels","marine","landscape","human","nature","wildlife", "engagement"]

In [74]:
avg_top_25 = pd.DataFrame(top_25[["marine","landscape","human","nature","wildlife"]].mean()).sort_values(0, ascending = False)
avg_top_25.columns = ["Top 25% Avg Topic Weight"]
avg_top_25

,Top 25% Avg Topic Weight
nature,0.252407
wildlife,0.242786
landscape,0.220671
human,0.162037
marine,0.122099


In [75]:
avg_bottom_25 = pd.DataFrame(bottom_25[["marine","landscape","human","nature","wildlife"]].mean()).sort_values(0, ascending = False)
avg_bottom_25.columns = ["Bottom 25% Avg Topic Weight"]
avg_bottom_25

,Bottom 25% Avg Topic Weight
human,0.395546
nature,0.190242
landscape,0.165360
wildlife,0.137848
marine,0.111004


# Task D

#### The confusion matrix in three logistic regression

In [114]:
print("Logistic Regression With Labels")
print("Out of Sample Accuracy:" , acc1)
pd.crosstab(y_test, y_pred, rownames = ["Actual Class"], colnames = ["Predicted Class"])

Logistic Regression With Labels
Out of Sample Accuracy: 0.7611111111111111


Predicted Class,0,1
Actual Class,,
0,67,20
1,23,70


In [115]:
print("Logistic Regression With Caption")
print("Out of Sample Accuracy:" , acc2)
pd.crosstab(y1_test, y_pred1, rownames = ["Actual Class"], colnames = ["Predicted Class"])

Logistic Regression With Caption
Out of Sample Accuracy: 0.7166666666666667


Predicted Class,0,1
Actual Class,,
0,61,26
1,25,68


In [116]:
print("Logistic Regression With Labels and Caption")
print("Out of Sample Accuracy:" , acc3)
pd.crosstab(y2_test, y_pred2, rownames = ["Actual Class"], colnames = ["Predicted Class"])

Logistic Regression With Labels and Caption
Out of Sample Accuracy: 0.7833333333333333


Predicted Class,0,1
Actual Class,,
0,65,22
1,17,76


In [117]:
avg_top_25

,Top 25% Avg Topic Weight
nature,0.252407
wildlife,0.242786
landscape,0.220671
human,0.162037
marine,0.122099


In [118]:
avg_bottom_25

,Bottom 25% Avg Topic Weight
human,0.395546
nature,0.190242
landscape,0.165360
wildlife,0.137848
marine,0.111004
